In [8]:
import tensorflow as tf # Make sure that python interpreter is 3.9.13 Global env
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wfdb
import ast


In [9]:
ptb = pd.read_csv('C:/Users/varun/ECG_Research_Project/ECG_Model/data/ptbxl_database.csv')
ptb.head()
len(ptb)

21799

In [98]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

In [96]:
path = 'C:/Users/varun/ECG_Research_Project/ECG_Model/data'
sampling_rate=100

In [100]:

# load and convert annotation data
Y = pd.read_csv(path+'/ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, 'C:/Users/varun/ECG_Research_Project/ECG_Model/data/')

(21799, 1000, 12)

In [102]:
temp = pd.DataFrame(X[1])
X.shape, Y.shape

((21799, 1000, 12), (21799, 27))

In [79]:
Y.loc[17]

patient_id                                                                13619.0
age                                                                          56.0
sex                                                                             0
height                                                                        NaN
weight                                                                        NaN
nurse                                                                         2.0
site                                                                          0.0
device                                                                  CS-12   E
recording_date                                                1985-01-20 14:25:25
report                          vorhof:hf  280 tachykardes vorhofflimmern uebe...
scp_codes                              {'AFLT': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
heart_axis                                                                   ALAD
infarction_stadi

In [103]:

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'/scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))


In [66]:
Y.strat_fold

ecg_id
1        3
2        2
3        5
4        3
5        4
        ..
21833    7
21834    4
21835    2
21836    8
21837    9
Name: strat_fold, Length: 21799, dtype: int64

In [186]:

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
test_fold =10
# Train
# X_train = X[]


X_train = X[(Y.strat_fold != test_fold)]
y_train = Y[Y.strat_fold != test_fold]

X_test = X[(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold]



X_train.shape, X_test.shape, y_train.shape, y_test.shape


((19601, 1000, 12), (2198, 1000, 12), (19601, 28), (2198, 28))